# Excel File Manipulation with Reader and Writer Packages

## The Reader and Writer Packages 

Almost every Excel file type requires a different package.

## When to Use Which Package 

When to use which package for different excel file format (Read/Write/Edit):
+ xlsx (OpenPyXL / OpenPyXL, XlsxWriter / OpenPyXL)
+ xlsm (OpenPyXL / OpenPyXL, XlsxWriter / OpenPyXL)
+ xltx,xltm (OpenPyXL / OpenPyXL / OpenPyXL)
+ xlsb (pyxlsb / - / -)
+ xls,xlt (xlrd / xlwt / xlutils)

High-level overview of where they differ:
+ OpenPyXL can read, write, and edit while XlsxWriter can only write.
+ OpenPyXL makes it easier to produce Excel files with VBA macros.
+ XlsxWriter is better documented.
+ XlsxWriter tends to be faster than OpenPyXL, but depending on the size of the workbook you’re writing, the differences may not be significant.

pandas uses the writer package it can find and if you have both OpenPyXL and XlsxWriter installed, XlsxWriter is the default. If you want to choose which package pandas should use, specify the engine parameter in the read_excel or to_excel functions or the ExcelFile and ExcelWriter classes, respectively. The engine is the package name in lower case.

## OpenPyXL 

### Reading with OpenPyXL

In [4]:
import pandas as pd
import openpyxl
import datetime as dt

In [68]:
filepath = originalpath + "xl/stores.xlsx"

In [69]:
# Open the workbook to read cell values.
# The file is automatically closed again after loading the data.
book = openpyxl.load_workbook(filepath, data_only=True)

In [9]:
# Get a worksheet object by name or index (0-based)
sheet = book["2019"]
sheet = book.worksheets[0]

In [11]:
# Get a list with all sheet names
book.sheetnames

['2019', '2020', '2019-2020']

In [12]:
# Loop through the sheet objects.
# Instead of "name", openpyxl uses "title".
for i in book.worksheets:
    print(i.title)

2019
2020
2019-2020


In [13]:
# Getting the dimensions,
# i.e., the used range of the sheet
sheet.max_row, sheet.max_column

(8, 6)

In [14]:
# Read the value of a single cell
# using "A1" notation and using cell indices (1-based)
sheet["B6"].value
sheet.cell(row=6, column=2).value

'Boston'

### Writing with OpenPyXL 

In [31]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Font, colors
from openpyxl.styles.borders import Border, Side
from openpyxl.styles.alignment import Alignment
from openpyxl.styles.fills import PatternFill
import excel

In [33]:
# Instantiate a workbook
book = openpyxl.Workbook()

# Get the first sheet and give it a name
sheet = book.active
sheet.title = "Sheet1"

# Writing individual cells using A1 notation
# and cell indices (1-based)
sheet["A1"].value = "Hello 1"
sheet.cell(row=2, column=1, value="Hello 2")

# Formatting: fill color, alignment, border and font
font_format = Font(color="FF0000", bold=True)
thin = Side(border_style="thin", color="FF0000")
sheet["A3"].value = "Hello3"
sheet["A3"].font = font_format
sheet["A3"].border = Border(top=thin, left=thin, right=thin, bottom=thin)
sheet["A3"].alignment = Alignment(horizontal="center")
sheet["A3"].fill = PatternFill(fgColor="FFFF00", fill_type="solid")

# Number formatting (using Excel's formatting strings)
sheet["A4"].value = 3.3333
sheet["A4"].number_format = "0.00"

# Date formatting (using Excel's formatting strings)
sheet["A5"].value = dt.date(2016, 10, 13)
sheet["A5"].number_format = "mm/dd/yy"

# Formula: you must use the English name of the formula
# with commas as delimiters
sheet["A6"].value = "=SUM(A4, 2)"

# Image
# image_path = r"/Users/wenjiazhu/Documents/Training/Data Analysis/python-for-excel/python-for-excel-1st-edition/images/python.png"
# sheet.add_image(Image(image_path), "C1")

# Two-dimensional list (we're using our excel module)
# data = [[None, "North", "South"],
#         ["Last Year", 2, 5],
#         ["This Year", 3, 6]]
# excel.write(sheet, data, "A10")

# Chart
chart = BarChart()
chart.type = "col"
chart.title = "Sales Per Region"
chart.x_axis.title = "Regions"
chart.y_axis.title = "Sales"
chart_data = Reference(sheet, min_row=11, min_col=1,
                        max_row=12, max_col=3)
chart_categories = Reference(sheet, min_row=10, min_col=2,
                        max_row=10, max_col=3)
# from_rows interprets the data in the same way
# as if you would add a chart manually in Excel
chart.add_data(chart_data, titles_from_data=True, from_rows=True)
chart.set_categories(chart_categories)
sheet.add_chart(chart, "A15")

# Saving the workbook creates the file on disk
book.save("openpyxl.xlsx")

If you want to write an Excel tempalte file, set the template attribute to True before saving it.

In [34]:
book = openpyxl.Workbook()
sheet = book.active
sheet["A1"].value = "This is a template"
book.template = True
book.save("template.xltx")

### Editing with OpenPyXL

In [35]:
filepath = orginalpath + "xl/stores.xlsx"

In [36]:
# Read the stores.xlsx file, change a cell
# and store it under a new location/name.
book = openpyxl.load_workbook(filepath)
book["2019"]["A1"].value = "modified"
book.save("stores_edited.xlsx")

In [37]:
filepath = originalpath + "xl/macro.xlsm"

In [38]:
book = openpyxl.load_workbook(filepath, keep_vba=True)
book["Sheet1"]["A1"].value = "Click the button!"
book.save("macro_openpyxl.xlsm")

## XlsxWriter 

As the name suggests, XlsxWriter can only write Excel files. Note that XlsxWriter uses zero-based cell indices, while Open‐ PyXL uses one-based cell indices.

In [40]:
import datetime as dt
import xlsxwriter
import excel

In [42]:
# Instantiate a workbook
book = xlsxwriter.Workbook("xlsxwriter.xlsx")

# Add a sheet and give it a name
sheet = book.add_worksheet("Sheet1")

# Writing individual cells using A1 notation
# and cell indices (0-based)
sheet.write("A1", "Hello 1")
sheet.write(1, 0, "Hello 2")

# Formatting: fill color, alignment, border and font
formatting = book.add_format({"font_color": "#FF0000",
                              "bg_color": "#FFFF00",
                              "bold": True, "align": "center",
                              "border": 1, "border_color": "#FF0000"})
sheet.write("A3", "Hello 3", formatting)

# Number formatting (using Excel's formatting strings)
number_format = book.add_format({"num_format": "0.00"})
sheet.write("A4", 3.3333, number_format)

# Date formatting (using Excel's formatting strings)
date_format = book.add_format({"num_format": "mm/dd/yy"})
sheet.write("A5", dt.date(2016, 10, 13), date_format)

# Formula: you must use the English name of the formula # with commas as delimiters
sheet.write("A6", "=SUM(A4, 2)")


# Image
image_path = originalpath + "images/python.png"
sheet.insert_image(0, 2, image_path)

# Two-dimensional list (we're using our excel module)
data = [[None, "North", "South"],
        ["Last Year", 2, 5],
        ["This Year", 3, 6]]
# excel.write(sheet, data, "A10")

# Chart: see the file "sales_report_xlsxwriter.py" in the # companion repo to see how you can work with indices
# instead of cell addresses
chart = book.add_chart({"type": "column"}) 
chart.set_title({"name": "Sales per Region"}) 
chart.add_series({"name": "=Sheet1!A11",
                  "categories": "=Sheet1!B10:C10",
                  "values": "=Sheet1!B11:C11"})
chart.add_series({"name": "=Sheet1!A12",
                  "categories": "=Sheet1!B10:C10",
                  "values": "=Sheet1!B12:C12"})
chart.set_x_axis({"name": "Regions"})
chart.set_y_axis({"name": "Sales"})
sheet.insert_chart("A15", chart)

# Closing the workbook creates the file on disk
book.close()

In comparison to OpenPyXL, XlsxWriter has to take a more complicated approach to write *xlsm* files - you need to extract the macro code from an existing Excel file to create a *.bin* file. Skip for now...

## pyxlsb 

In [43]:
import pyxlsb
import excel

In [44]:
filepath = originalpath + "xl/stores.xlsb"

In [45]:
# Loop through sheets. With pyxlsb, the workbook
# and sheet objects can be used as context managers.
# book.sheets returns a list of sheet names, not objects! 
# To get a sheet object, use get_sheet() instead.
with pyxlsb.open_workbook(filepath) as book:
    for sheet_name in book.sheets:
        with book.get_sheet(sheet_name) as sheet:
            dim = sheet.dimension 
            print(f"Sheet '{sheet_name}' has "
                  f"{dim.h} rows and {dim.w} cols")

Sheet '2019' has 7 rows and 5 cols
Sheet '2020' has 7 rows and 5 cols
Sheet '2019-2020' has 20 rows and 5 cols


In [49]:
# Read in the values of a range of cells by using our excel module. 
# Instead of "2019", you could also use its index (1-based).
with pyxlsb.open_workbook(filepath) as book:
    with book.get_sheet("2019") as sheet: 
#         data = excel.read(sheet, "B2")
# data[:2] 
# Print the first two rows
        for row in sheet.rows():
            print(row)

[Cell(r=0, c=0, v=None), Cell(r=0, c=1, v=None), Cell(r=0, c=2, v=None), Cell(r=0, c=3, v=None), Cell(r=0, c=4, v=None), Cell(r=0, c=5, v=None)]
[Cell(r=1, c=0, v=None), Cell(r=1, c=1, v='Store'), Cell(r=1, c=2, v='Employees'), Cell(r=1, c=3, v='Manager'), Cell(r=1, c=4, v='Since'), Cell(r=1, c=5, v='Flagship')]
[Cell(r=2, c=0, v=None), Cell(r=2, c=1, v='New York'), Cell(r=2, c=2, v=10.0), Cell(r=2, c=3, v='Sarah'), Cell(r=2, c=4, v=43301.0), Cell(r=2, c=5, v=False)]
[Cell(r=3, c=0, v=None), Cell(r=3, c=1, v='San Francisco'), Cell(r=3, c=2, v=12.0), Cell(r=3, c=3, v='Neriah'), Cell(r=3, c=4, v=43771.0), Cell(r=3, c=5, v='MISSING')]
[Cell(r=4, c=0, v=None), Cell(r=4, c=1, v='Chicago'), Cell(r=4, c=2, v=4.0), Cell(r=4, c=3, v='Katelin'), Cell(r=4, c=4, v=43861.0), Cell(r=4, c=5, v=None)]
[Cell(r=5, c=0, v=None), Cell(r=5, c=1, v='Boston'), Cell(r=5, c=2, v=5.0), Cell(r=5, c=3, v='Georgiana'), Cell(r=5, c=4, v=42826.0), Cell(r=5, c=5, v=True)]
[Cell(r=6, c=0, v=None), Cell(r=6, c=1, v='Wa

In [50]:
df = pd.read_excel(filepath)
df.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,Store,Employees,Manager,Since,Flagship
1,NaN,New York,10,Sarah,43301,False
2,NaN,San Francisco,12,Neriah,43771,MISSING
3,NaN,Chicago,4,Katelin,43861,NaN
4,NaN,Boston,5,Georgiana,42826,True


## xlrd, xlwt, and xlutils

The combination of xlrd, xlwt, and xlutils offers roughly the same functionality for the legacy xls format that OpenPyXL offers for the xlsx format: 
+ xlrd reads, 
+ xlwt writes, and 
+ xlutils edits xls files. 

These packages aren’t actively developed anymore, but they are likely going to be relevant as long as there are still xls files around. 

### Reading with xlrd

In [52]:
import xlrd 
import xlwt
from xlwt.Utils import cell_to_rowcol2 
import xlutils

In [53]:
filepath = originalpath + "xl/stores.xls"

In [55]:
# Open the workbook to read cell values. The file is 
# automatically closed again after loading the data.
book = xlrd.open_workbook(filepath)

In [56]:
# Get a list with all sheet names 
book.sheet_names()

['2019', '2020', '2019-2020']

In [57]:
# Loop through the sheet objects 
for sheet in book.sheets():
    print(sheet.name)

2019
2020
2019-2020


In [58]:
# Get a sheet object by name or index (0-based) 
sheet = book.sheet_by_index(0)
sheet = book.sheet_by_name("2019")

In [59]:
# Dimensions
sheet.nrows, sheet.ncols

(8, 6)

In [60]:
# Read the value of a single cell
# using "A1" notation and using cell indices (0-based). 
# The "*" unpacks the tuple that cell_to_rowcol2 returns 
# into individual arguments. 
sheet.cell(*cell_to_rowcol2("B3")).value
sheet.cell(2, 1).value

'New York'

### Writing with xlwt

In [61]:
import xlwt
from xlwt.Utils import cell_to_rowcol2 
import datetime as dt

In [63]:
# Instantiate a workbook 
book = xlwt.Workbook()

# Add a sheet and give it a name
sheet = book.add_sheet("Sheet1")

# Writing individual cells using A1 notation 
# and cell indices (0-based) 
sheet.write(*cell_to_rowcol2("A1"), "Hello 1") 
sheet.write(r=1, c=0, label="Hello 2")

# Formatting: fill color, alignment, border and font
formatting = xlwt.easyxf("font: bold on, color red;"
                         "align: horiz center;"
                         "borders: top_color red, bottom_color red,"
                                  "right_color red, left_color red,"
                                  "left thin, right thin,"
                                  "top thin, bottom thin;"
                         "pattern: pattern solid, fore_color yellow;")
sheet.write(r=2, c=0, label="Hello 3", style=formatting)

# Number formatting (using Excel's formatting strings)
number_format = xlwt.easyxf(num_format_str="0.00")
sheet.write(3, 0, 3.3333, number_format)

# Date formatting (using Excel's formatting strings)
date_format = xlwt.easyxf(num_format_str="mm/dd/yyyy")
sheet.write(4, 0, dt.datetime(2012, 2, 3), date_format)

# Formula: you must use the English name of the formula 
# with commas as delimiters
sheet.write(5, 0, xlwt.Formula("SUM(A4, 2)"))

# Two-dimensional list (we're using our excel module)
data = [[None, "North", "South"],
        ["Last Year", 2, 5],
        ["This Year", 3, 6]]
# excel.write(sheet, data, "A10")

 # Picture (only allows to add bmp format)
imagepath = originalpath + "images/python.bmp"
sheet.insert_bitmap(imagepath, 0, 2)

# This writes the file to disk
book.save("xlwt.xls")

### Editing with xlutils 

xlutils acts as a bridge between xlrd and xlwt. This makes it explicit that this is not a true editing operation: the spreadsheet is read including the formatting via xlrd (by setting formatting_info=True) and then written out again by xlwt, including the changes that were made in between:

In [64]:
import xlutils.copy

In [65]:
filepath = originalpath + "xl/stores.xls"

In [66]:
book = xlrd.open_workbook(filepath, formatting_info=True)
book = xlutils.copy.copy(book)
book.save("stores_edited.xls")

# Advanced Reader and Writer Topics

Learn how to use pandas together with the reader and writer packages.

## Working with Big Excel Files 

### Writing with OpenPyXL 

When writing large files with OpenPyXL, make sure to have the lxml package installed, as this makes the writing process faster. The critical option, though, is the **write_only=True** flag, which makes sure that the memory consumption remains low. It, however, forces you to write row by row by using the append method and won’t allow you to write single cells anymore:

In [70]:
book = openpyxl.Workbook(write_only=True)
# With write_only=True, book.active doesn't work 
sheet = book.create_sheet()
# This will produce a sheet with 1000 x 200 cells 
for row in range(1000):
    sheet.append(list(range(200)))
book.save("openpyxl_optimized.xlsx")


### Writing with XlsxWriter

XlsxWriter has a similar option like OpenPyXL called **constant_memory**. It forces you to write sequential rows, too.

In [71]:
book = xlsxwriter.Workbook("xlsxwriter_optimized.xlsx",
                            options={"constant_memory": True})
sheet = book.add_worksheet()
# This will produce a sheet with 1000 x 200 cells 
for row in range(1000):
    sheet.write_row(row , 0, list(range(200)))
book.close()

### Reading with xlrd

When reading big files in the legacy xls format, xlrd allows you to load sheets on demand, like this:

In [74]:
filepath = originalpath + "xl/stores.xls"

In [76]:
with xlrd.open_workbook(filepath, on_demand=True) as book: 
    sheet = book.sheet_by_index(0) # Only loads the first sheet

If you wouldn’t use the workbook as a context manager as we do here, you would need to call **book.release_resources()** manually to properly close the workbook again. To use xlrd in this mode with pandas, use it like this:

In [77]:
with xlrd.open_workbook(filepath, on_demand=True) as book: 
    with pd.ExcelFile(book, engine="xlrd") as f:
        df = pd.read_excel(f, sheet_name=0)

### Reading with OpenPyXL 

To keep memory under control when reading big Excel files with OpenPyXL, you should load the workbook with **read_only=True**. Since OpenPyXL doesn’t support the with statement, you will need to make sure to close the file again when you’re done. If your file contains links to external workbooks, you may additionally want to use **keep_links=False** to make it faster.

In [78]:
filepath = originalpath + "xl/big.xlsx"
book = openpyxl.load_workbook(filepath,
                              data_only=True, read_only=True,
                              keep_links=False)
# Perform the desired read operations here
book.close() # Required with read_only=True

In [80]:
book.sheetnames

['Sheet1', 'Sheet2', 'Sheet3', 'Sheet4', 'Sheet5']

## Formatting DataFrames in Excel 

In [81]:
filepath = originalpath + "xl/stores.xlsx"

with pd.ExcelFile(filepath, engine="openpyxl") as xlfile:
    # Read a DataFrame
    df = pd.read_excel(xlfile, sheet_name="2020")
    
    # Get the OpenPyXL workbook object
    book = xlfile.book
    
    # From here on, it's OpenPyXL code
    sheet = book["2019"]
    value = sheet["B3"].value # Read a single value

In [82]:
with pd.ExcelWriter("pandas_and_openpyxl.xlsx",
                    engine="openpyxl") as writer:
    df = pd.DataFrame({"col1": [1, 2, 3, 4], "col2": [5, 6, 7, 8]})
    # Write a DataFrame
    df.to_excel(writer, "Sheet1", startrow=4, startcol=2)
    
    # Get the OpenPyXL workbook and sheet objects
    book = writer.book
    sheet = writer.sheets["Sheet1"]
    
    # From here on, it's OpenPyXL code
    sheet["A1"].value = "This is a Title" # Write a single cell value

### Formatting a DataFrame's index and headers 

In [83]:
df = pd.DataFrame({"col1": [1, -2], "col2": [-3, 4]},
                 index = ["row1", "row2"])
df.index.name = "ix"
df

,col1,col2
ix,,
row1,1,-3
row2,-2,4


In [84]:
from openpyxl.styles import PatternFill

In [86]:
with pd.ExcelWriter("formatting_openpyxl.xlsx",
                   engine="openpyxl") as writer:
    # Write out the df with the default formatting to A1
    df.to_excel(writer, startrow=0, startcol=0)
    
    # Write out the df with custom index/header formatting to A6
    startrow, startcol = 0, 5
    
    # 1. Write out the data part of the DataFrame
    df.to_excel(writer, header=False, index=False, 
               startrow=startrow+1, startcol=startcol+1)
    # Get the sheet object and create a style object
    sheet = writer.sheets["Sheet1"]
    style = PatternFill(fgColor="D9D9D9", fill_type="solid")
    
    # 2. Write out the styled column headers
    for i, col in enumerate(df.columns):
        sheet.cell(row=startrow+1, column=i+startcol+2, value=col).fill = style
    
    # 3. Write out the styled index
    index = [df.index.name if df.index.name else None] + list(df.index)
    for i, row in enumerate(index):
        sheet.cell(row=i+startrow+1, column=startcol+1, value=row).fill = style

In [88]:
# Formatting index/headers with XlsxWriter
with pd.ExcelWriter("formatting_xlsxwriter.xlsx",
                   engine="xlsxwriter") as writer:
    # Write out the df with the default formatting to A1
    df.to_excel(writer, startrow=0, startcol=0)
    
    # Write out the df with custom index/header formatting to A6
    startrow, startcol = 0, 5
    
    # 1. Write out the data part of the DataFrame
    df.to_excel(writer, header=False, index=False,
                startrow=startrow + 1, startcol=startcol + 1)
    # Get the book and sheet object and create a style object 
    book = writer.book
    sheet = writer.sheets["Sheet1"]
    style = book.add_format({"bg_color": "#D9D9D9"})
    
    # 2. Write out the styled column headers
    for i, col in enumerate(df.columns): 
        sheet.write(startrow, startcol + i + 1, col, style)

    # 3. Write out the styled index
    index = [df.index.name if df.index.name else None] + list(df.index) 
    for i, row in enumerate(index):
        sheet.write(startrow + i, startcol, row, style)

### Formatting a DataFrame's data part 

OpenPyXL can apply a format to each cell, while XlsxWriter can only apply formats on a row or column basis. 

In [89]:
from openpyxl.styles import Alignment

In [90]:
with pd.ExcelWriter("data_format_openpyxl.xlsx", engine="openpyxl") as writer:
    # Write out the DataFrame
    df.to_excel(writer)
                 
    # Get the book and sheet objects
    book = writer.book
    sheet = writer.sheets["Sheet1"]
    
    # Formatting individual cells
    nrows, ncols = df.shape 
    for row in range(nrows):
        for col in range(ncols):
            # +1 to account for the header/index 
            # +1 since OpenPyXL is 1-based
            cell = sheet.cell(row=row + 2,
                              column=col + 2)
            cell.number_format = "0.000"
            cell.alignment = Alignment(horizontal="center")

In [91]:
with pd.ExcelWriter("data_format_xlsxwriter.xlsx", engine="xlsxwriter") as writer:
    # Write out the DataFrame
    df.to_excel(writer)
    
    # Get the book and sheet objects
    book = writer.book
    sheet = writer.sheets["Sheet1"]
    
    # Formatting the columns (individual cells can't be formatted)
    number_format = book.add_format({"num_format": "0.000",
                                     "align": "center"})
    sheet.set_column(first_col=1, last_col=2,
                     cell_format=number_format)

As an alternative, pandas offers *experimental support* for the style property of DataFrames. To apply the same format as in the previous example (three decimals and center align), you’ll need to apply a function to every element of a Styler object via **applymap**. You get a Styler object via the df.style attribute:

In [92]:
df.style.applymap(lambda x: "number-format: 0.000;" "text-align: center")\
    .to_excel("styled.xlsx")

In [93]:
df = pd.DataFrame({"Date": [dt.date(2020, 1, 1)],
                   "Datetime": [dt.datetime(2020, 1, 1, 10)]})
with pd.ExcelWriter("date.xlsx",
                    date_format="yyyy-mm-dd",
                    datetime_format="yyyy-mm-dd hh:mm:ss") as writer:
    df.to_excel(writer)